<a href="https://colab.research.google.com/github/sindhu819/Spark-2020/blob/main/Copy_of_Spark_vs_Pandas_for_large_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Examining performance of Spark vs. Pandas on large datasets**

To run spark in Colab, we need to first install pyspark


In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [75.1 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 k

Congrats! Your Colab is ready to run Pyspark. 

# EDA

The goal of this exercise to explore and analyze the efficency of Spark vs. Pandas in importing and analyzing data

In [ ]:
import pandas as pd

In [ ]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## We will import a dataset of around 25 million rows in this exercise ##

In [ ]:
%%time
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'1iD3sjlz23qvbW2sry_WpS5cINWwIINEv'}) # replace the id with id of file you want to access
downloaded.GetContentFile('50store_data1to6.csv')

CPU times: user 2.61 s, sys: 4.69 s, total: 7.29 s
Wall time: 15.2 s


### Importing csv data into dataframes

In [ ]:
%%time
dataset = spark.read.csv('50store_data1to6.csv',inferSchema=True, header =False)

CPU times: user 7.2 ms, sys: 2.63 ms, total: 9.83 ms
Wall time: 41.4 s


In [ ]:
%%time
data_pd = pd.read_csv('50store_data1to6.csv',header=None)

CPU times: user 18.6 s, sys: 1.71 s, total: 20.3 s
Wall time: 20.3 s


### Reading 3 rows of data

In [ ]:
%%time
#The Spark way
dataset.show(3)

+---+---+----------------+--------------+-----+--------------------+
|_c0|_c1|             _c2|           _c3|  _c4|                 _c5|
+---+---+----------------+--------------+-----+--------------------+
|P11|  1|2016/04/12 18:29|Oil - Avocado1|13.15|Raising Cane's Re...|
|P12|  1|2016/04/12 18:29|Oil - Avocado2|13.15|Raising Cane's Re...|
|P13|  1|2016/04/12 18:29|Oil - Avocado3|13.15|Raising Cane's Re...|
+---+---+----------------+--------------+-----+--------------------+
only showing top 3 rows

CPU times: user 2.17 ms, sys: 73 µs, total: 2.24 ms
Wall time: 363 ms


In [ ]:
%%time
#The Pandas way
data_pd.head(3)

CPU times: user 569 µs, sys: 0 ns, total: 569 µs
Wall time: 560 µs


,0,1,2,3,4,5
0,P11,1,2016/04/12 18:29,Oil - Avocado1,13.15,Raising Cane's Restaurants
1,P12,1,2016/04/12 18:29,Oil - Avocado2,13.15,Raising Cane's Restaurants
2,P13,1,2016/04/12 18:29,Oil - Avocado3,13.15,Raising Cane's Restaurants


### Counting the size of the dataframes (how many rows and columns in this dataframe)

In [ ]:
%%time
# the Spark way
print(dataset.count())
print(len(dataset.columns))

24770000
6
CPU times: user 1.74 ms, sys: 1.4 ms, total: 3.14 ms
Wall time: 12.5 s


In [ ]:
%%time
#the Pandas way
data_pd.count()

CPU times: user 3.88 s, sys: 9.12 ms, total: 3.89 s
Wall time: 3.89 s


0    24665000
1    24665000
2    24665000
3    24665000
4    24665000
5    24665000
dtype: int64

### Calculating the overall numeric represenation using describe()

In [ ]:
%%time
#the Spark way
dataset.describe().show()

+-------+--------+------------------+----------------+------------------+------------------+--------------------+
|summary|     _c0|               _c1|             _c2|               _c3|               _c4|                 _c5|
+-------+--------+------------------+----------------+------------------+------------------+--------------------+
|  count|24770000|          24770000|        24770000|          24770000|          24770000|            24770000|
|   mean|    null|              25.5|            null|              null|26.725815502647993|                null|
| stddev|    null|14.430869980959411|            null|              null|33.542871668988454|                null|
|    min|  P10001|                 1|2016/04/12 18:29|      7up  355 Ml1|              0.13|Burkett Restauran...|
|    max|   P9999|                50|2016/04/21 03:30|Zucchini - Yellow9|             469.8|     ddFoodSolutions|
+-------+--------+------------------+----------------+------------------+---------------

In [ ]:
%%time
#the Pandas way
data_pd.describe()

CPU times: user 1.38 s, sys: 10.7 ms, total: 1.39 s
Wall time: 1.4 s


,1,4
count,2.466500e+07,2.466500e+07
mean,2.550000e+01,2.672121e+01
std,1.443087e+01,3.355578e+01
min,1.000000e+00,1.300000e-01
25%,1.300000e+01,6.230000e+00
50%,2.550000e+01,1.677000e+01
75%,3.800000e+01,3.360000e+01
max,5.000000e+01,4.698000e+02


### Looking for any null values in all columns

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

In [ ]:
%%time
# the Spark approach

dataset.select([count(when(isnan(c), c)).alias(c) for c in dataset.columns]).show()

+---+---+---+---+---+---+
|_c0|_c1|_c2|_c3|_c4|_c5|
+---+---+---+---+---+---+
|  0|  0|  0|  0|  0|  0|
+---+---+---+---+---+---+

CPU times: user 51.6 ms, sys: 18.7 ms, total: 70.3 ms
Wall time: 4min 3s


In [ ]:
%%time
# the Pandas approach
data_pd.isnull().sum()

CPU times: user 3.45 s, sys: 697 µs, total: 3.45 s
Wall time: 3.46 s


0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64

### Counting how many distinct values the second column has and how many occurences are in each distinct value

In [ ]:
%%time
# the Spark approach
dataset.select("_c1").distinct().count()

CPU times: user 6.45 ms, sys: 4.07 ms, total: 10.5 ms
Wall time: 34.6 s


50

In [ ]:
dataset.groupby("_c1").count().show()

+---+------+
|_c1| count|
+---+------+
| 31|495400|
| 34|495400|
| 28|495400|
| 26|495400|
| 27|495400|
| 44|495400|
| 12|495400|
| 22|495400|
| 47|495400|
|  1|495400|
| 13|495400|
|  6|495400|
| 16|495400|
|  3|495400|
| 20|495400|
| 40|495400|
| 48|495400|
|  5|495400|
| 19|495400|
| 41|495400|
+---+------+
only showing top 20 rows



In [ ]:
%%time
# the Pandas approach
data_pd.groupby(1).count()

CPU times: user 4.96 s, sys: 13.8 ms, total: 4.98 s
Wall time: 4.98 s


,0,2,3,4,5
1,,,,,
1,493300,493300,493300,493300,493300
2,493300,493300,493300,493300,493300
3,493300,493300,493300,493300,493300
4,493300,493300,493300,493300,493300
5,493300,493300,493300,493300,493300
6,493300,493300,493300,493300,493300
7,493300,493300,493300,493300,493300
8,493300,493300,493300,493300,493300
9,493300,493300,493300,493300,493300


### Filtering data with second column as P11

In [ ]:
%%time
# the Spark approach with SQL
dataset.createOrReplaceTempView("dataset")
spark.sql("select * from dataset where _c0 ='P11'").show()

+---+---+----------------+--------------+-----+--------------------+
|_c0|_c1|             _c2|           _c3|  _c4|                 _c5|
+---+---+----------------+--------------+-----+--------------------+
|P11|  1|2016/04/12 18:29|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/13 04:56|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/14 14:33|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/15 13:06|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/16 13:42|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/17 10:36|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/18 05:13|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/19 17:04|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/20 07:13|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/21 03:30|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  2|2016/04/12 18:29|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  2|2016/04/13 04:56|Oil - Av

In [ ]:
dataset.filter(dataset._c0=="P11").show()

+---+---+----------------+--------------+-----+--------------------+
|_c0|_c1|             _c2|           _c3|  _c4|                 _c5|
+---+---+----------------+--------------+-----+--------------------+
|P11|  1|2016/04/12 18:29|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/13 04:56|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/14 14:33|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/15 13:06|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/16 13:42|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/17 10:36|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/18 05:13|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/19 17:04|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/20 07:13|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/21 03:30|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  2|2016/04/12 18:29|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  2|2016/04/13 04:56|Oil - Av

In [ ]:
%%time
# the Spark approach with filtering
dataset.filter(col("_c0")=="P11").show()

+---+---+----------------+--------------+-----+--------------------+
|_c0|_c1|             _c2|           _c3|  _c4|                 _c5|
+---+---+----------------+--------------+-----+--------------------+
|P11|  1|2016/04/12 18:29|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/13 04:56|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/14 14:33|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/15 13:06|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/16 13:42|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/17 10:36|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/18 05:13|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/19 17:04|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/20 07:13|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  1|2016/04/21 03:30|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  2|2016/04/12 18:29|Oil - Avocado1|13.15|Raising Cane's Re...|
|P11|  2|2016/04/13 04:56|Oil - Av

In [ ]:
%%time
# the Pandas approach
data_pd.loc[data_pd[0]=='P11']

CPU times: user 1.09 s, sys: 1.94 ms, total: 1.09 s
Wall time: 1.09 s


,0,1,2,3,4,5
0,P11,1,2016/04/12 18:29,Oil - Avocado1,13.15,Raising Cane's Restaurants
49330,P11,1,2016/04/13 04:56,Oil - Avocado1,13.15,Raising Cane's Restaurants
98660,P11,1,2016/04/14 14:33,Oil - Avocado1,13.15,Raising Cane's Restaurants
147990,P11,1,2016/04/15 13:06,Oil - Avocado1,13.15,Raising Cane's Restaurants
197320,P11,1,2016/04/16 13:42,Oil - Avocado1,13.15,Raising Cane's Restaurants
...,...,...,...,...,...,...
24418350,P11,50,2016/04/17 10:36,Oil - Avocado1,13.15,Raising Cane's Restaurants
24467680,P11,50,2016/04/18 05:13,Oil - Avocado1,13.15,Raising Cane's Restaurants
24517010,P11,50,2016/04/19 17:04,Oil - Avocado1,13.15,Raising Cane's Restaurants
24566340,P11,50,2016/04/20 07:13,Oil - Avocado1,13.15,Raising Cane's Restaurants


In [ ]:
data_pd[data_pd.iloc[:,0]=='P11']

,0,1,2,3,4,5
0,P11,1,2016/04/12 18:29,Oil - Avocado1,13.15,Raising Cane's Restaurants
49330,P11,1,2016/04/13 04:56,Oil - Avocado1,13.15,Raising Cane's Restaurants
98660,P11,1,2016/04/14 14:33,Oil - Avocado1,13.15,Raising Cane's Restaurants
147990,P11,1,2016/04/15 13:06,Oil - Avocado1,13.15,Raising Cane's Restaurants
197320,P11,1,2016/04/16 13:42,Oil - Avocado1,13.15,Raising Cane's Restaurants
...,...,...,...,...,...,...
24418350,P11,50,2016/04/17 10:36,Oil - Avocado1,13.15,Raising Cane's Restaurants
24467680,P11,50,2016/04/18 05:13,Oil - Avocado1,13.15,Raising Cane's Restaurants
24517010,P11,50,2016/04/19 17:04,Oil - Avocado1,13.15,Raising Cane's Restaurants
24566340,P11,50,2016/04/20 07:13,Oil - Avocado1,13.15,Raising Cane's Restaurants
